In [ ]:
# Copyright 2020, Gurobi Optimization, LLC

# Solve a network flow problem. 
# One products is produced in 2 cities ('Detroit' and 'Denver') and must be sent to
# warehouses in 3 cities ('Boston', 'New York', and 'Seattle') to
# satisfy demand ('inflow[i]').
#
# Flows on the transportation network must respect arc capacity constraints
# ('capacity[i,j]'). The objective is to minimize the sum of the arc
# transportation costs ('cost[i,j]').

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Base data
nodes = ['Detroit', 'Denver', 'Boston', 'NewYork', 'Seattle']

In [ ]:
arcs, capacity = gp.multidict({
    ('Detroit', 'Boston'):   100,
    ('Detroit', 'NewYork'):  80,
    ('Detroit', 'Seattle'):  120,
    ('Denver',  'Boston'):   120,
    ('Denver',  'NewYork'): 120,
    ('Denver',  'Seattle'):  120})

In [ ]:
# Cost for pairs source-destination
cost = {
    ('Detroit', 'Boston'):   10,
    ('Detroit', 'NewYork'): 20,
    ('Detroit', 'Seattle'):  60,
    ('Denver',  'Boston'):   40,
    ('Denver',  'NewYork'): 40,
    ('Denver',  'Seattle'):  30 }

In [ ]:
# Demand for city
demand = {
    ('Detroit'):   50,
    ('Denver'):    60,
    ('Boston'):   -50,
    ('NewYork'): -50,
    ('Seattle'):  -10 }

In [ ]:
# Create optimization model
m = gp.Model('cost_min_flow')

In [ ]:
# Create variables
x = m.addVars(arcs, obj=cost, name="x")

In [ ]:
# Arc-capacity constraints
m.addConstrs((x[i, j] <= capacity[i, j] for i, j in arcs), "cap")

In [ ]:
# Flow-conservation constraints
m.addConstrs((x.sum(j, '*') - x.sum('*', j) == demand[j] for j in nodes), "node")

In [ ]:
# write problem
m.write('cost_min_flow.lp')

In [ ]:
# Compute optimal solution
m.optimize()

In [ ]:
# Print solution
if m.status == GRB.OPTIMAL:
    print('Optimal solution: %g' % m.objVal)
    solution = m.getAttr('x', x)
    print('\nOptimal network flows:')
    for i, j in arcs:
        if solution[i, j] > 0:
            print('%s -> %s: %g' % (i, j, solution[i, j]))